# Setup

In [75]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

!pip install pillow==4.1.1
%reload_ext autoreload
%autoreload  

# Get training and validation images



In [76]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Load a zip image file by ID and unzip its contents.
dataFile = drive.CreateFile({'id': '1MWvJ0zpc_JL0VauYHB4JiOajQWKm-7p7'}).GetContentFile('flower_data.zip')
!unzip 'flower_data.zip'

# 3. Load the label file.
dataFile = drive.CreateFile({'id': '1nb19NViylUq9fZLia5faLm0_AJfky7Rj'}).GetContentFile('cat_to_name.json')

Archive:  flower_data.zip
replace flower_data/valid/61/image_06296.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='https://github.com/seetd/pytorch_challenge/blob/master/assets/Flowers.png?raw=1' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [0]:
%matplotlib inline

# Imports here
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as ds
import torchvision.transforms as transforms
import torchvision.models as models
from time import gmtime, process_time, strftime
from collections import OrderedDict
import os
import json
from PIL import Image

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [0]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
# TODO: Define your transforms for the training and validation sets
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(255),
        transforms.RandomResizedCrop(224),
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ]),
    'eval': transforms.Compose([
        transforms.Resize(255),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ]),
}

# TODO: Load the datasets with ImageFolder
datasets = {
    'train': ds.ImageFolder(train_dir, transform=data_transforms['train']),
    'valid': ds.ImageFolder(valid_dir, transform=data_transforms['eval'])
}

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
    'train': DataLoader(datasets['train'], batch_size=30, shuffle=True),
    'valid': DataLoader(datasets['valid'], batch_size=30, shuffle=True)    
}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [0]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

## Saving Checkpoint file to Google Drive

In [0]:
def save_to_gdrive(filename):
  # Create & upload a text file.
  uploaded = drive.CreateFile()
  uploaded.SetContentFile(filename)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [0]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(dataset, model, optimizer, filename='checkpoint.pth.tar'):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 0
    valid_loss_min = np.Inf
    class_to_idx = dataset.class_to_idx
    
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        valid_loss_min = checkpoint['loss']
        class_to_idx = checkpoint['class_to_idx']        
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})".format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return start_epoch, valid_loss_min, class_to_idx

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [0]:
# TODO: Save the checkpoint 
def save_checkpoint(epoch, loss, dataset, model, optimizer, filename='checkpoint.pth.tar'):
  state = {
    'epoch': epoch,
    'loss': loss,
    'class_to_idx': dataset.class_to_idx,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
  }
  torch.save(state, filename)


## Model

In [84]:
# TODO: Build and train your network
model = models.densenet161(pretrained=True)

# Freeze the parameters so we don't backprop through them as we are only replacing the classifer
# Classifier will be replaced later
for param in model.parameters():
    param.requires_grad = False

classifier = nn.Linear(2208, 102);
model.classifier = classifier

# Move the model to GPU if supported
model.to(device)

# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.classifier.parameters(), lr=0.01)

#dropout = nn.Dropout(0.5)
#classifier = nn.Sequential(OrderedDict([
#                              ('fc1', nn.Linear(25088, 8192)),
#                              ('relu', nn.ReLU()),
#                              ('dropout', dropout),
#                              ('fc2', nn.Linear(8192, 2048)),
#                              ('relu', nn.ReLU()),
#                              ('dropout', dropout),    
#                              ('fc3', nn.Linear(2048, 102)),
#                              ('output', nn.LogSoftmax(dim=1))
#                          ]))
# model.classifier = classifier

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [0]:
def train(epochs, last_start_epoch, last_valid_loss_min):
  start = process_time()
  valid_loss_min = last_valid_loss_min
  
  print('Training start time in {}(GMT)'.format(strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())))
  for epoch in range(last_start_epoch, epochs):
    epoch_start = process_time()
    # monitor training loos
    train_loss = 0.0
    valid_loss = 0.0
    
    # train the model
    model.train()
    for data, target in dataloaders["train"]:
      # Move data and target to GPU if supported
      data, target = data.to(device), target.to(device)
      # clear the gradients of all optimized variables
      optimizer.zero_grad()
      # forward pass: compute predicted outputs by passing inputs to the model
      output = model(data)
      # calculate the loss
      loss = criterion(output, target)
      # backward pass: compute gradient of the loss with respect to the model parameters
      loss.backward()
      # perform a single optimization step (parameter update)
      optimizer.step()
      # update running training loss
      train_loss += loss.item() * data.size(0)
        
    # validate the model      
    model.eval()
    for data, target in dataloaders["valid"]:
      # Move data and target to GPU if supported
      data, target = data.to(device), target.to(device)
      # forward pass: compute predicted outputs by passing inputs to the model
      output = model(data)        
      # Calculate the loss
      loss = criterion(output, target)
      # update running validation loss
      valid_loss += loss.item() * data.size(0)
        
      
    # print training/ validation statistics
    # calculate average loss over an epoch
    train_loss = train_loss/len(dataloaders["train"].dataset)
    valid_loss = valid_loss/len(dataloaders["valid"].dataset)
    
    print('Epoch: {}\tTraining Loss: {:.6f}\tValidation Loss: {:.6f}\tTime taken:{}'.format(epoch + 1, train_loss, valid_loss, process_time() - epoch_start))
    # save model if validation loss has decreased
    if valid_loss < valid_loss_min:
      print('Validation loss decreased by ({:.6f}). Saving model ... '.format(valid_loss_min - valid_loss))
      save_checkpoint(epoch, loss, dataloaders["train"].dataset, model, optimizer, filename)
      print('Model saved.')
      valid_loss_min = valid_loss  

  end = process_time()
  print('Training end time in {}(GMT). Taken {} seconds'.format(strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()), end - start))

In [89]:
# number of epochs to train the model
epochs = 50

# initialize tracker for minimum validation loss by setting it to infinity
filename = 'densenet161.ptr.tar'
start_epoch, valid_loss_min, class_to_idx = load_checkpoint(dataloaders["train"].dataset, model, optimizer, filename)
save_checkpoint(start_epoch, valid_loss_min, dataloaders["train"].dataset, model, optimizer, filename)
train(epochs, start_epoch, valid_loss_min)

save_to_gdrive(filename)

=> loading checkpoint 'densenet161.ptr.tar'
=> loaded checkpoint 'densenet161.ptr.tar' (epoch 0)
Training start time in Thu, 20 Dec 2018 03:23:31 +0000(GMT)
Epoch: 1	Training Loss: 2.577453	Validation Loss: 2.191159	Time taken:233.19656863499995
Validation loss decreased by (0.452590). Saving model ... 
Model saved.
Epoch: 2	Training Loss: 2.154868	Validation Loss: 1.844044	Time taken:232.29846271700012
Validation loss decreased by (0.347115). Saving model ... 
Model saved.
Epoch: 3	Training Loss: 1.868983	Validation Loss: 1.617749	Time taken:231.66849949499988
Validation loss decreased by (0.226295). Saving model ... 
Model saved.
Epoch: 4	Training Loss: 1.658525	Validation Loss: 1.388039	Time taken:231.13728400100013
Validation loss decreased by (0.229710). Saving model ... 
Model saved.
Epoch: 5	Training Loss: 1.504670	Validation Loss: 1.225976	Time taken:230.57449007600007
Validation loss decreased by (0.162063). Saving model ... 
Model saved.


KeyboardInterrupt: ignored

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [0]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    tensor_image = data_transforms['valid'](image)
    return tensor_image.unsqueeze_(0)    

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [0]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [0]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    # TODO: Implement the code to predict the class from an image file
    pil_image = Image.open(image_path)
    tensor_image = process_image(pil_image).to(device)
    result = model(tensor_image)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='https://github.com/seetd/pytorch_challenge/blob/master/assets/inference_example.png?raw=1' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [0]:
# TODO: Display an image along with the top 5 classes